In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split, WeightedRandomSampler, SubsetRandomSampler
import os
from tqdm import tqdm
from PIL import Image
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold
import torchvision.transforms as transforms

torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class GalaxyDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted([d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))])
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        self.images = []
        self.labels = []
        
        # Walk through all subdirectories and collect image paths and labels
        for class_name in self.classes:
            class_dir = os.path.join(root_dir, class_name)
            class_idx = self.class_to_idx[class_name]
            
            for img_name in os.listdir(class_dir):
                if img_name.endswith(('.png', '.jpg', '.jpeg')):
                    self.images.append(os.path.join(class_dir, img_name))
                    self.labels.append(class_idx)
        
        # Calculate class weights
        label_counter = Counter(self.labels)
        self.class_weights = {cls: 1.0/count for cls, count in label_counter.items()}
        
        # Store sample weights for WeightedRandomSampler
        self.sample_weights = [self.class_weights[label] for label in self.labels]
        self.sample_weights = torch.FloatTensor(self.sample_weights)
        
        # Print dataset statistics
        print(f"Dataset: {root_dir}")
        print(f"Classes: {self.classes}")
        print(f"Total images: {len(self.images)}")
        print(f"Class distribution: {label_counter}")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        # Load and preprocess image
        image = Image.open(img_path).convert("RGB")
        
        # Apply transformations
        if self.transform:
            image = self.transform(image)
        else:
            # Default preprocessing if no transform provided
            image = image.resize((128, 128))  # Resize to power of 2 for better performance
            
            # Basic augmentation for training
            if self.root_dir.endswith('train'):
                if torch.rand(1) > 0.5:  # Random horizontal flip
                    image = image.transpose(Image.FLIP_LEFT_RIGHT)
                if torch.rand(1) > 0.5:  # Random vertical flip
                    image = image.transpose(Image.FLIP_TOP_BOTTOM)
            
            # Convert to numpy array and normalize
            image_array = np.array(image) / 255.0
            
            # Convert to tensor
            image = torch.FloatTensor(image_array).permute(2, 0, 1)
        
        return image, label


class GalaxyResNet(nn.Module):
    def __init__(self, num_classes):
        super(GalaxyResNet, self).__init__()
        
        # Initial convolution layer
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Residual blocks
        self.layer1 = self._make_layer(64, 64, 2)
        self.layer2 = self._make_layer(64, 128, 2, stride=2)
        self.layer3 = self._make_layer(128, 256, 2, stride=2)
        self.layer4 = self._make_layer(256, 512, 2, stride=2)
        
        # Classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(512, num_classes)
        
        # Initialize weights
        self._initialize_weights()
    
    def _make_layer(self, in_channels, out_channels, blocks, stride=1):
        layers = []
        
        # First block with potential stride
        layers.append(self._residual_block(in_channels, out_channels, stride))
        
        # Subsequent blocks
        for _ in range(1, blocks):
            layers.append(self._residual_block(out_channels, out_channels))
        
        return nn.Sequential(*layers)
    
    def _residual_block(self, in_channels, out_channels, stride=1):
        downsample = None
        if stride != 1 or in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
        
        return ResidualBlock(in_channels, out_channels, stride, downsample)
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        
        return x


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
    
    def forward(self, x):
        identity = x
        
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
        
        out += identity
        out = self.relu(out)
        
        return out


class EarlyStopping:
    def __init__(self, patience=5, verbose=False, delta=0.01, path='checkpoint.pt'):
        self.patience = patience
        self.verbose = verbose
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score - self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Save model when validation loss decreases.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


def train_fold(model, train_loader, val_loader, criterion, optimizer, scheduler, early_stopping, num_epochs, fold):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []

    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        progress_bar = tqdm(train_loader, desc=f'Fold {fold+1} - Epoch {epoch+1}/{num_epochs}')
        
        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            progress_bar.set_postfix({'loss': running_loss / len(train_loader), 'accuracy': 100. * correct / total})
        
        train_loss = running_loss / len(train_loader)
        train_accuracy = 100. * correct / total
        train_losses.append(train_loss)
        train_accuracies.append(train_accuracy)
        
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
        
        val_loss /= len(val_loader)
        val_accuracy = 100. * val_correct / val_total
        val_losses.append(val_loss)
        val_accuracies.append(val_accuracy)
        
        print(f'\nFold {fold+1} - Epoch {epoch+1}/{num_epochs}: '
              f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, '
              f'Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
        
        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break
        
        scheduler.step(val_accuracy)

    # Load the best model
    model.load_state_dict(torch.load(early_stopping.path))
    
    # Plot metrics for this fold
    plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, fold)
    
    return model


def plot_metrics(train_losses, val_losses, train_accuracies, val_accuracies, fold):
    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot losses
    ax1.plot(train_losses, label='Train Loss')
    ax1.plot(val_losses, label='Validation Loss')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.set_title(f'Fold {fold+1} - Training & Validation Loss')
    
    # Plot accuracies
    ax2.plot(train_accuracies, label='Train Accuracy')
    ax2.plot(val_accuracies, label='Validation Accuracy')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Accuracy (%)')
    ax2.legend()
    ax2.set_title(f'Fold {fold+1} - Training & Validation Accuracy')
    
    plt.tight_layout()
    plt.savefig(f'metrics_fold_{fold+1}.png')
    plt.close()


def evaluate_model(model, loader, class_names, fold=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for inputs, labels in tqdm(loader, desc=f'Evaluating Fold {fold+1}' if fold is not None else 'Evaluating'):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100. * correct / total
    print(f'Accuracy: {accuracy:.2f}%')
    
    num_classes = len(class_names)
    conf_matrix = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    title = f'Fold {fold+1} - Confusion Matrix' if fold is not None else 'Confusion Matrix'
    plt.title(title)
    plt.tight_layout()
    plt.savefig(f'confusion_matrix_fold_{fold+1}.png' if fold is not None else 'confusion_matrix.png')
    plt.close()
    
    report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
    print("\nClassification Report:\n", report)
    
    # Save classification report to file
    with open(f'classification_report_fold_{fold+1}.txt' if fold is not None else 'classification_report.txt', 'w') as f:
        f.write(report)

    return accuracy, all_preds, all_labels


def k_fold_cross_validation(train_dataset, num_folds=5, num_epochs=30):
    results = {}
    all_fold_accuracies = []
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    
    num_classes = len(train_dataset.classes)
    class_names = train_dataset.classes
    
    # Get the class weights for loss function
    label_counter = Counter(train_dataset.labels)
    class_weights = torch.FloatTensor([1.0/label_counter[i] for i in range(num_classes)]).to(device)
    
    for fold, (train_idx, val_idx) in enumerate(kfold.split(train_dataset)):
        print(f'\n{"="*20} Fold {fold+1}/{num_folds} {"="*20}')
        
        # Sample validation indices directly
        train_sampler = SubsetRandomSampler(train_idx)
        val_sampler = SubsetRandomSampler(val_idx)
        
        train_loader = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)
        val_loader = DataLoader(train_dataset, batch_size=32, sampler=val_sampler)
        
        model = GalaxyResNet(num_classes=num_classes).to(device)
        criterion = nn.CrossEntropyLoss(weight=class_weights)
        optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True, min_lr=1e-6)
        early_stopping = EarlyStopping(patience=5, verbose=True, delta=0.01, path=f'model_fold_{fold+1}.pt')
        
        model = train_fold(model, train_loader, val_loader, criterion, optimizer, scheduler, early_stopping, num_epochs, fold)
        accuracy, predictions, labels = evaluate_model(model, val_loader, class_names, fold)
        
        results[fold] = {
            'accuracy': accuracy,
            'model_path': f'model_fold_{fold+1}.pt'
        }
        all_fold_accuracies.append(accuracy)
    
    # Calculate and print average accuracy across all folds
    avg_accuracy = sum(all_fold_accuracies) / len(all_fold_accuracies)
    print(f'\nAverage accuracy across {num_folds} folds: {avg_accuracy:.2f}%')
    
    # Find the best fold
    best_fold = max(results.keys(), key=lambda k: results[k]['accuracy'])
    print(f'Best fold: {best_fold+1} with accuracy: {results[best_fold]["accuracy"]:.2f}%')
    
    # Save the best model as the final model
    best_model_path = results[best_fold]['model_path']
    torch.save(torch.load(best_model_path), 'galaxy_cnn_final.pth')
    print(f'Best model saved as: galaxy_cnn_final.pth')
    
    return results


def test_ensemble_model(models, test_loader, class_names):
    """Test with an ensemble of models by averaging predictions"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    # Set all models to evaluation mode
    for model in models:
        model.eval()
    
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Testing ensemble model'):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Get predictions from all models
            outputs_sum = None
            for model in models:
                outputs = model(inputs)
                if outputs_sum is None:
                    outputs_sum = outputs
                else:
                    outputs_sum += outputs
            
            # Average predictions
            outputs_avg = outputs_sum / len(models)
            _, predicted = outputs_avg.max(1)
            
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    # Calculate accuracy
    accuracy = 100. * correct / total
    print(f'Ensemble Model Accuracy: {accuracy:.2f}%')
    
    # Create confusion matrix
    conf_matrix = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Ensemble Model - Confusion Matrix')
    plt.tight_layout()
    plt.savefig('ensemble_confusion_matrix.png')
    plt.close()
    
    # Generate classification report
    report = classification_report(all_labels, all_preds, target_names=class_names, digits=4)
    print("\nEnsemble Model Classification Report:\n", report)
    
    # Save classification report to file
    with open('ensemble_classification_report.txt', 'w') as f:
        f.write(report)
    
    return accuracy


def main():
    # Dataset paths
    train_dir = "./galaxy_dataset_rgb/train"
    test_dir = "./galaxy_dataset_rgb/test"
    
    # Define augmentation transform for training
    train_transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Define transformation for validation/test
    test_transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Create datasets
    train_dataset = GalaxyDataset(root_dir=train_dir, transform=train_transform)
    test_dataset = GalaxyDataset(root_dir=test_dir, transform=test_transform)
    
    # Create test dataloader
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
    # Number of classes
    num_classes = len(train_dataset.classes)
    class_names = train_dataset.classes
    print(f"Number of classes: {num_classes}")
    print(f"Class names: {class_names}")
    
    # Perform k-fold cross validation
    num_folds = 5
    results = k_fold_cross_validation(train_dataset, num_folds=num_folds, num_epochs=30)
    
    # Create ensemble model (from all folds)
    ensemble_models = []
    for fold in range(num_folds):
        model = GalaxyResNet(num_classes=num_classes).to(device)
        model.load_state_dict(torch.load(f'model_fold_{fold+1}.pt'))
        ensemble_models.append(model)
    
    # Test the ensemble model
    ensemble_accuracy = test_ensemble_model(ensemble_models, test_loader, class_names)
    
    # Also test the best individual model
    best_fold = max(results.keys(), key=lambda k: results[k]['accuracy'])
    best_model = GalaxyResNet(num_classes=num_classes).to(device)
    best_model.load_state_dict(torch.load('galaxy_cnn_final.pth'))
    print("\nEvaluating the best individual model on test set:")
    best_model_accuracy, _, _ = evaluate_model(best_model, test_loader, class_names)
    
    print("\nResults Summary:")
    print(f"Best Individual Model (Fold {best_fold+1}) Accuracy on Test Set: {best_model_accuracy:.2f}%")
    print(f"Ensemble Model Accuracy on Test Set: {ensemble_accuracy:.2f}%")
    
    print("Training and evaluation completed!")


if __name__ == '__main__':
    main()

Using device: cuda
Dataset: ./galaxy_dataset_rgb/train
Classes: ['Early_Spiral', 'Elliptical', 'Irregular', 'Late_Spiral', 'Lenticular']
Total images: 32731
Class distribution: Counter({2: 28793, 3: 1996, 4: 801, 0: 648, 1: 493})
Dataset: ./galaxy_dataset_rgb/test
Classes: ['Early_Spiral', 'Elliptical', 'Irregular', 'Late_Spiral', 'Lenticular']
Total images: 8183
Class distribution: Counter({2: 7198, 3: 499, 4: 201, 0: 162, 1: 123})
Number of classes: 5
Class names: ['Early_Spiral', 'Elliptical', 'Irregular', 'Late_Spiral', 'Lenticular']

==================== Fold 1/5 ====================


c:\Users\Aseem\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
Fold 1 - Epoch 1/30: 100%|██████████| 819/819 [01:39<00:00,  8.23it/s, loss=1.51, accuracy=41.1] 



Fold 1 - Epoch 1/30: Train Loss: 1.5113, Train Acc: 41.07%, Val Loss: 1.4002, Val Acc: 46.16%
Validation loss decreased (inf --> 1.400184). Saving model...


Fold 1 - Epoch 2/30: 100%|██████████| 819/819 [01:29<00:00,  9.14it/s, loss=1.38, accuracy=41.7] 



Fold 1 - Epoch 2/30: Train Loss: 1.3825, Train Acc: 41.65%, Val Loss: 1.3139, Val Acc: 30.76%
Validation loss decreased (1.400184 --> 1.313912). Saving model...


...

Accuracy: 52.35%

Classification Report:
               precision    recall  f1-score   support

Early_Spiral     0.1062    0.5512    0.1781       127
  Elliptical     0.0970    0.6699    0.1695       103
   Irregular     0.9605    0.4990    0.6567      5748
 Late_Spiral     0.2452    0.8396    0.3796       399
  Lenticular     0.1032    0.5030    0.1712       169

    accuracy                         0.5235      6546
   macro avg     0.3024    0.6125    0.3110      6546
weighted avg     0.8646    0.5235    0.6104      6546


Average accuracy across 5 folds: 59.29%
Best fold: 2 with accuracy: 76.46%
Best model saved as: galaxy_cnn_final.pth


Testing ensemble model: 100%|██████████| 256/256 [00:23<00:00, 10.81it/s]


Ensemble Model Accuracy: 70.02%

Ensemble Model Classification Report:
               precision    recall  f1-score   support

Early_Spiral     0.2248    0.5926    0.3260       162
  Elliptical     0.1023    0.7967    0.1813       123
   Irregular     0.9803    0.6920    0.8113      7198
 Late_Spiral     0.4569    0.8397    0.5918       499
  Lenticular     0.1700    0.6766    0.2717       201

    accuracy                         0.7002      8183
   macro avg     0.3869    0.7195    0.4364      8183
weighted avg     0.9003    0.7002    0.7656      8183


Evaluating the best individual model on test set:


Evaluating: 100%|██████████| 256/256 [00:16<00:00, 15.14it/s]


Accuracy: 74.46%

Classification Report:
               precision    recall  f1-score   support

Early_Spiral     0.2402    0.6790    0.3548       162
  Elliptical     0.1131    0.8211    0.1988       123
   Irregular     0.9843    0.7403    0.8451      7198
 Late_Spiral     0.5544    0.8277    0.6640       499
  Lenticular     0.2080    0.6965    0.3204       201

    accuracy                         0.7446      8183
   macro avg     0.4200    0.7529    0.4766      8183
weighted avg     0.9112    0.7446    0.8017      8183


Results Summary:
Best Individual Model (Fold 2) Accuracy on Test Set: 74.46%
Ensemble Model Accuracy on Test Set: 70.02%
Training and evaluation completed!
